In [2]:
import numpy as np
import pandas as pd

covid=pd.read_csv("../input/corona-virus-report/covid_19_clean_complete.csv")
covid.head(10)

In [11]:
# 5.1) Line chart
reg  = covid.loc[((covid['WHO Region'] == 'Eastern Mediterranean') |
                  (covid['WHO Region'] == 'Americas') |
                  (covid['WHO Region'] == 'Europe')),:]
reg = reg.groupby(['Date','WHO Region'],dropna=False)['Deaths'].sum().to_frame()
reg = reg.pivot_table('Deaths','Date','WHO Region').plot(figsize=(10,6),
                                                         legend=True, rot=90,
                                                         title = 'Mortes por Região',
                                                         xlabel = 'Data',
                                                         grid = 'On',
                                                         ylabel = 'Deaths')

# A região que teve mais mortes foi a das Américas
# Todas as Regiões começam a subir entre 12/03/2020 e 06/04/2020
# A mais acentuada foi a européia, entretando há uma redução do aumento de mortes,
# nas outras duas regiões não há redução na acentuação, mas a do leste do mediterrâneo
# a acentuação é visualmente menor que as demais, resultando no menor número de mortes.

In [13]:
# 5.2) Bar chart

wmd=pd.read_csv("../input/corona-virus-report/worldometer_data.csv")
wmdF = wmd.drop(['TotalCases','NewCases','TotalDeaths','NewDeaths','TotalRecovered',
                'NewRecovered','ActiveCases','Serious,Critical','Tot Cases/1M pop',
                'Deaths/1M pop','TotalTests','Tests/1M pop','WHO Region'],axis=1)

prov = covid.groupby(['Province/State','Country/Region'], dropna=False)['Deaths'].max()
paises = prov.groupby(['Country/Region']).sum()
covidPpais = wmdF.merge(paises, on='Country/Region')
covidPcont = covidPpais.groupby(['Continent']).sum()
covidPcont = covidPcont.drop('Population',axis=1)
covidPcont.plot.bar(figsize=(10,6),
                    legend = False,
                    title = 'Mortes por continente',
                    xlabel = 'Continente',
                    ylabel = 'Deaths')

# Os 2 continentes com o maior número de mortes são a américa do sul e a europa.
# O continente com menos mortes é a Oceania.
# O descaso com a pandemia foi maior nesses dois continentes, embora a Europa tenha 
# adotado mais medidas de contenção tardiamente.

In [26]:
wmd

In [56]:
# 5.3) Scatter Plots

mmpp = wmd.groupby(['Continent'])['Deaths/1M pop'].mean().nlargest(3)
mmpp = list(mmpp.index.values)

mmppdf = wmd.loc[((wmd['Continent'] == mmpp[0]) |
                  (wmd['Continent'] == mmpp[1]) |
                  (wmd['Continent'] == mmpp[2])),:]
mmppdf = mmppdf[['Continent','Deaths/1M pop','Population']]
mmppdf = mmppdf.pivot_table('Population','Deaths/1M pop','Continent')

c0 = mmppdf[[mmpp[0]]].dropna(axis=0).reset_index()
c1 = mmppdf[[mmpp[1]]].dropna(axis=0).reset_index()
c2 = mmppdf[[mmpp[2]]].dropna(axis=0).reset_index()

ax1 = c0.plot(kind='scatter', x='Deaths/1M pop', y=mmpp[0], color='r',label=mmpp[0])    
ax2 = c1.plot(kind='scatter', x='Deaths/1M pop', y=mmpp[1], color='g',ax=ax1,label=mmpp[1])    
ax3 = c2.plot(kind='scatter', x='Deaths/1M pop', y=mmpp[2], color='b',
              ax=ax1,label=mmpp[2],legend=True,figsize=(10,6),
              ylabel = 'População',xlabel = 'Mortes por milhão de hab.',
              title='População por Mortes',grid = 'On')

# A maioria dos dados se encontra abaixo de 5*10^7 habitantes e
# abaixo de 600 mortes por milhão de habitantes.
# Não é possível notar correlação entre a população e o n° de hab.
# Não é possível notar relação entre as colunas exceto a comentada no primeiro item.

In [88]:
# 5.4) Heatmap de Correlação
import seaborn as sns
import matplotlib.pyplot as plt

nwmd = wmd.select_dtypes(['number'])
corr = nwmd.corr(method='pearson')

plt.figure(dpi=300)
plt.title('Correlações dos dados numéricos')
sns.heatmap(corr, annot=True, fmt='.2f',annot_kws={'fontsize': 8})

bdcor = wmd[['Tests/1M pop','Tot Cases/1M pop']]
corr2 = bdcor.corr(method='pearson')

plt.figure(dpi=300)
plt.title('Correlação entre mortes por milhão e população')
sns.heatmap(corr2, annot=True, fmt='.2f')

In [59]:
wmd